## 데이터 로드
* https://data.seoul.go.kr/dataVisual/seoul/seoulLivingPopulation.do
* https://data.seoul.go.kr/dataList/OA-14991/S/1/datasetView.do?tab=A

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv

df = pd.read_csv('LOCAL_PEOPLE_20251123.csv', encoding='cp949', engine='python', on_bad_lines='skip')
df

In [ ]:
df.info()

In [ ]:
"1" + "1"

In [ ]:
1 + 1

In [ ]:
# 문자와 숫자가 섞여있으면 계산이 안 됩니다. 숫자는 수치 데이터 형태로 변경해 주어야 합니다.
# 1 + "1"

In [ ]:
# 문자 데이터를 수치 데이터로 변경
numerical_cols = df.columns.drop(['기준일ID', '시간대구분', '행정동코드'])

for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.info()

In [ ]:
df.describe()

## 깔끔한 데이터 만들기

In [ ]:
id_vars = ['기준일ID', '시간대구분', '행정동코드', '집계구코드']
value_vars = [col for col in df.columns if col not in id_vars]

df_tidy = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='변수', value_name='생활인구수')
df_tidy.head()

## 파생변수 만들기

In [ ]:
df_tidy['성별'] = df_tidy['변수'].apply(lambda x: '남' if '남자' in x else ('여' if '여자' in x else 'Total'))
df_tidy['연령대'] = df_tidy['변수'].apply(lambda x: x.replace('남자', '').replace('여자', '').replace('생활인구수', '') if '생활인구수' in x else x)
df_tidy.loc[df_tidy['연령대'] == '총', '연령대'] = '총합'

df_tidy = df_tidy.drop(columns=['변수'])

df_tidy.head()

In [ ]:
df_tidy['생활인구구분'] = df_tidy['성별'].apply(lambda x: '총생활인구수' if x == 'Total' else 'Age/Gender Specific Population')

print("'생활인구구분' column created.")

print(df_tidy.info())
df_tidy.describe()

In [ ]:
df_tidy.describe(include='object')

In [ ]:
df_tidy

## 한글 폰트 설정

In [ ]:
!pip install -Uq koreanize-matplotlib
import koreanize_matplotlib

In [ ]:
df_tidy.hist(figsize=(10, 10), bins=50);

## 행정동 코드 매핑

In [ ]:
mapping_df = pd.read_excel('/content/행정동코드_매핑정보_20241218.xlsx', skiprows=[0])
display(mapping_df.head())
mapping_df.info()

In [ ]:
mapping_df[['H_DNG_CD', 'H_DNG_NM']].set_index('H_DNG_CD')['H_DNG_NM'].to_dict()

In [ ]:
H_DNG_CD = mapping_df[['H_DNG_CD', 'H_DNG_NM']].set_index('H_DNG_CD')['H_DNG_NM'].to_dict()
H_DNG_CD[11140670]

In [ ]:
df_tidy['행정동코드'] = df_tidy['행정동코드'].astype(int)
df_tidy['행정동명'] = df_tidy['행정동코드'].map(H_DNG_CD)
df_tidy.head()

In [ ]:
df_tidy['행정동명'].unique()

In [ ]:
len(df_tidy['행정동명'].unique())

In [ ]:
df_tidy['행정동명'].value_counts()

In [ ]:
df_tidy['연령대'].value_counts()

## 교차표와 피봇테이블

In [ ]:
hour_age = pd.crosstab(df_tidy['시간대구분'], df_tidy['연령대'], values=df_tidy['생활인구수'], aggfunc='sum')
hour_age

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(hour_age.iloc[:, :-1], annot=True, cmap='Blues', fmt=',.0f')

## 특정 행정동 분석

In [ ]:
df_tidy['행정동명'].unique()

In [ ]:
df_filtered = df_tidy[df_tidy['행정동명'].isin(['을지로동', '명동', '목1동', '성수1가1동'])]
df_filtered = df_filtered[df_filtered['연령대'] != '총합']
df_filtered.head()

In [ ]:
plt.figure(figsize=(20, 5))
sns.pointplot(data=df_filtered, x='연령대', y='생활인구수', hue='행정동명')

In [ ]:
plt.figure(figsize=(20, 5))
sns.pointplot(data=df_filtered, x='시간대구분', y='생활인구수', hue='행정동명')

## 특정동

In [ ]:
df_yeonnam_dong = df_tidy[df_tidy['행정동명'] == '연남동']
df_yeonnam_dong = df_yeonnam_dong[df_yeonnam_dong['연령대'] != '총합']

df_yeonnam_dong.head()

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(yeonnam_dong_crosstab, annot=True, cmap='Blues', fmt='.0f', linewidths=.5)
plt.title('연남동 시간대별 연령대별 생활인구수 (히트맵)')
plt.xlabel('연령대')
plt.ylabel('시간대구분')
plt.show()

In [ ]:
df_yeonnam_dong_melted = yeonnam_dong_crosstab.reset_index().melt(id_vars='시간대구분', var_name='연령대', value_name='생활인구수')
df_yeonnam_dong_melted.head()

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot(data=df_yeonnam_dong_melted, x='시간대구분', y='생활인구수', hue='연령대', palette='viridis')
plt.title('연남동 시간대별 연령대별 생활인구수 (바 차트)')
plt.xlabel('시간대구분')
plt.ylabel('생활인구수')
plt.legend(title='연령대', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()